In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🧪 Laboratorio 5: Criptografía Asimétrica — RSA\n",
    "\n",
    "## 📚 Contenido\n",
    "1. Algoritmo de Euclides y Euclides Extendido\n",
    "2. Exponenciación Modular Rápida (Square-and-Multiply)\n",
    "3. Generación de Claves RSA\n",
    "4. Cifrado y Descifrado RSA\n",
    "5. Aplicación RSA con Alfabeto A-Z\n",
    "6. Casos de Prueba y Validación\n",
    "\n",
    "---\n",
    "\n",
    "## 🎯 Objetivos\n",
    "\n",
    "- Implementar el **Algoritmo de Euclides** y su versión extendida\n",
    "- Calcular **inversos modulares** de forma eficiente\n",
    "- Implementar **exponenciación modular rápida** (crítica para RSA)\n",
    "- Construir un **sistema RSA completo** de extremo a extremo\n",
    "- Aplicar RSA para cifrar **mensajes de texto** usando bloques\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1️⃣ Algoritmo de Euclides y Euclides Extendido\n",
    "\n",
    "### 📖 Teoría\n",
    "\n",
    "El **Algoritmo de Euclides** calcula el Máximo Común Divisor (MCD) de dos números:\n",
    "\n",
    "$$\\gcd(a, b) = \\gcd(b, a \\mod b)$$\n",
    "\n",
    "El **Algoritmo de Euclides Extendido** encuentra coeficientes $x, y$ tales que:\n",
    "\n",
    "$$a \\cdot x + b \\cdot y = \\gcd(a, b)$$\n",
    "\n",
    "Esta identidad (de Bézout) es fundamental para calcular inversos modulares."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def gcd(a: int, b: int) -> int:\n",
    "    \"\"\"\n",
    "    Calcula el Máximo Común Divisor usando el Algoritmo de Euclides.\n",
    "    \n",
    "    Args:\n",
    "        a, b: Números enteros\n",
    "    \n",
    "    Returns:\n",
    "        MCD(a, b)\n",
    "    \n",
    "    Ejemplo:\n",
    "        >>> gcd(48, 18)\n",
    "        6\n",
    "    \"\"\"\n",
    "    while b != 0:\n",
    "        a, b = b, a % b\n",
    "    return abs(a)\n",
    "\n",
    "# Pruebas\n",
    "print(\"=== Algoritmo de Euclides ===\")\n",
    "print(f\"gcd(48, 18) = {gcd(48, 18)}\")\n",
    "print(f\"gcd(17, 3120) = {gcd(17, 3120)}\")\n",
    "print(f\"gcd(100, 35) = {gcd(100, 35)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def extended_gcd(a: int, b: int) -> tuple[int, int, int]:\n",
    "    \"\"\"\n",
    "    Algoritmo de Euclides Extendido.\n",
    "    Retorna (gcd, x, y) tal que a*x + b*y = gcd(a, b)\n",
    "    \n",
    "    Args:\n",
    "        a, b: Números enteros\n",
    "    \n",
    "    Returns:\n",
    "        Tupla (gcd, x, y)\n",
    "    \n",
    "    Ejemplo:\n",
    "        >>> extended_gcd(17, 3120)\n",
    "        (1, -367, 2)\n",
    "    \"\"\"\n",
    "    if b == 0:\n",
    "        return abs(a), 1 if a >= 0 else -1, 0\n",
    "    \n",
    "    # Recursión\n",
    "    gcd_val, x1, y1 = extended_gcd(b, a % b)\n",
    "    \n",
    "    # Actualizar coeficientes usando las relaciones de Bézout\n",
    "    x = y1\n",
    "    y = x1 - (a // b) * y1\n",
    "    \n",
    "    return gcd_val, x, y\n",
    "\n",
    "# Pruebas\n",
    "print(\"\\n=== Algoritmo de Euclides Extendido ===\")\n",
    "g, x, y = extended_gcd(17, 3120)\n",
    "print(f\"extended_gcd(17, 3120) = ({g}, {x}, {y})\")\n",
    "print(f\"Verificación: 17*{x} + 3120*{y} = {17*x + 3120*y}\")\n",
    "\n",
    "g, x, y = extended_gcd(48, 18)\n",
    "print(f\"\\nextended_gcd(48, 18) = ({g}, {x}, {y})\")\n",
    "print(f\"Verificación: 48*{x} + 18*{y} = {48*x + 18*y}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def modular_inverse(a: int, m: int) -> int:\n",
    "    \"\"\"\n",
    "    Calcula el inverso modular de 'a' módulo 'm'.\n",
    "    Encuentra x tal que (a * x) % m = 1\n",
    "    \n",
    "    Args:\n",
    "        a: Número a invertir\n",
    "        m: Módulo\n",
    "    \n",
    "    Returns:\n",
    "        Inverso modular de a módulo m\n",
    "    \n",
    "    Raises:\n",
    "        ValueError: Si no existe inverso (gcd(a,m) ≠ 1)\n",
    "    \n",
    "    Ejemplo:\n",
    "        >>> modular_inverse(17, 3120)\n",
    "        2753\n",
    "    \"\"\"\n",
    "    gcd_val, x, _ = extended_gcd(a, m)\n",
    "    \n",
    "    if gcd_val != 1:\n",
    "        raise ValueError(f\"No existe inverso: gcd({a}, {m}) = {gcd_val} ≠ 1\")\n",
    "    \n",
    "    # Asegurar que el resultado sea positivo\n",
    "    return x % m\n",
    "\n",
    "# Pruebas\n",
    "print(\"\\n=== Inverso Modular ===\")\n",
    "inv = modular_inverse(17, 3120)\n",
    "print(f\"modular_inverse(17, 3120) = {inv}\")\n",
    "print(f\"Verificación: (17 * {inv}) % 3120 = {(17 * inv) % 3120}\")\n",
    "\n",
    "inv2 = modular_inverse(7, 26)\n",
    "print(f\"\\nmodular_inverse(7, 26) = {inv2}\")\n",
    "print(f\"Verificación: (7 * {inv2}) % 26 = {(7 * inv2) % 26}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "\n",
    "## 2️⃣ Exponenciación Modular Rápida\n",
    "\n",
    "### 📖 Teoría\n",
    "\n",
    "El algoritmo **Square-and-Multiply** calcula $b^e \\mod m$ eficientemente:\n",
    "\n",
    "1. Convertir $e$ a binario\n",
    "2. Procesar cada bit de izquierda a derecha:\n",
    "   - **Siempre**: Elevar al cuadrado el resultado\n",
    "   - **Si bit = 1**: Multiplicar por la base\n",
    "\n",
    "**Complejidad**: $O(\\log e)$ multiplicaciones en lugar de $O(e)$\n",
    "\n",
    "**Ejemplo**: $5^{13} \\mod 7$\n",
    "- $13_{10} = 1101_2$\n",
    "- Bits: 1, 1, 0, 1"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def modular_exponentiation(base: int, exp: int, mod: int) -> int:\n",
    "    \"\"\"\n",
    "    Exponenciación modular rápida: base^exp mod mod\n",
    "    Algoritmo: Square-and-Multiply\n",
    "    \n",
    "    Args:\n",
    "        base: Base\n",
    "        exp: Exponente (debe ser >= 0)\n",
    "        mod: Módulo (debe ser > 0)\n",
    "    \n",
    "    Returns:\n",
    "        base^exp mod mod\n",
    "    \n",
    "    Ejemplo:\n",
    "        >>> modular_exponentiation(5, 13, 7)\n",
    "        3\n",
    "    \"\"\"\n",
    "    if mod == 1:\n",
    "        return 0\n",
    "    \n",
    "    result = 1\n",
    "    base = base % mod\n",
    "    \n",
    "    while exp > 0:\n",
    "        # Si el bit actual es 1, multiplicar por base\n",
    "        if exp & 1:\n",
    "            result = (result * base) % mod\n",
    "        \n",
    "        # Elevar base al cuadrado\n",
    "        base = (base * base) % mod\n",
    "        \n",
    "        # Desplazar exponente (dividir entre 2)\n",
    "        exp >>= 1\n",
    "    \n",
    "    return result\n",
    "\n",
    "# Pruebas\n",
    "print(\"=== Exponenciación Modular Rápida ===\")\n",
    "print(f\"5^13 mod 7 = {modular_exponentiation(5, 13, 7)}\")\n",
    "print(f\"65^17 mod 3233 = {modular_exponentiation(65, 17, 3233)}\")\n",
    "print(f\"2^10 mod 1000 = {modular_exponentiation(2, 10, 1000)}\")\n",
    "\n",
    "# Comparar con pow() de Python\n",
    "assert modular_exponentiation(5, 13, 7) == pow(5, 13, 7)\n",
    "assert modular_exponentiation(65, 17, 3233) == pow(65, 17, 3233)\n",
    "print(\"\\n✓ Todas las verificaciones con pow() pasaron exitosamente\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 🔍 Visualización del Algoritmo"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def modular_exponentiation_traced(base: int, exp: int, mod: int) -> int:\n",
    "    \"\"\"\n",
    "    Versión con trazabilidad para propósitos educativos.\n",
    "    Muestra cada paso del algoritmo.\n",
    "    \"\"\"\n",
    "    print(f\"\\nCalculando {base}^{exp} mod {mod}\")\n",
    "    print(\"=\"*70)\n",
    "    \n",
    "    # Mostrar exponente en binario\n",
    "    exp_binary = bin(exp)[2:]\n",
    "    print(f\"Exponente: {exp} = {exp_binary} (binario)\")\n",
    "    print(f\"\\n{'Paso':<6} {'Bit':<6} {'Operación':<40} {'Resultado':<10}\")\n",
    "    print(\"-\"*70)\n",
    "    \n",
    "    result = 1\n",
    "    base = base % mod\n",
    "    temp_exp = exp\n",
    "    step = 0\n",
    "    \n",
    "    # Obtener todos los bits\n",
    "    bits = []\n",
    "    while temp_exp > 0:\n",
    "        bits.append(temp_exp & 1)\n",
    "        temp_exp >>= 1\n",
    "    bits.reverse()\n",
    "    \n",
    "    # Procesar primer bit (siempre 1)\n",
    "    result = base\n",
    "    print(f\"{step+1:<6} {1:<6} result = base = {base}{' '*26}{result:<10}\")\n",
    "    step += 1\n",
    "    \n",
    "    # Procesar resto de bits\n",
    "    for bit in bits[1:]:\n",
    "        # Siempre elevar al cuadrado\n",
    "        old_result = result\n",
    "        result = (result * result) % mod\n",
    "        print(f\"{step+1:<6} {'-':<6} result = {old_result}² mod {mod}{' '*16}{result:<10}\")\n",
    "        step += 1\n",
    "        \n",
    "        # Si bit es 1, multiplicar por base\n",
    "        if bit:\n",
    "            old_result = result\n",
    "            result = (result * base) % mod\n",
    "            print(f\"{step+1:<6} {1:<6} result = {old_result} × {base} mod {mod}{' '*14}{result:<10}\")\n",
    "            step += 1\n",
    "    \n",
    "    print(\"-\"*70)\n",
    "    print(f\"\\n✓ Resultado: {base}^{exp} mod {mod} = {result}\\n\")\n",
    "    return result\n",
    "\n",
    "# Ejemplo con trazabilidad\n",
    "result = modular_exponentiation_traced(5, 13, 7)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "\n",
    "## 3️⃣ Generación de Claves RSA\n",
    "\n",
    "### 📖 Teoría\n",
    "\n",
    "**Pasos para generar claves RSA**:\n",
    "\n",
    "1. Elegir dos primos distintos $p$ y $q$\n",
    "2. Calcular $n = p \\times q$ (módulo público)\n",
    "3. Calcular $\\phi(n) = (p-1)(q-1)$ (función de Euler)\n",
    "4. Elegir $e$ tal que $1 < e < \\phi(n)$ y $\\gcd(e, \\phi(n)) = 1$\n",
    "5. Calcular $d = e^{-1} \\mod \\phi(n)$ (inverso modular)\n",
    "\n",
    "**Claves**:\n",
    "- **Pública**: $(e, n)$\n",
    "- **Privada**: $(d, n)$"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_phi(p: int, q: int) -> int:\n",
    "    \"\"\"\n",
    "    Calcula φ(n) = (p-1)(q-1) donde n = p*q\n",
    "    \n",
    "    Args:\n",
    "        p, q: Números primos\n",
    "    \n",
    "    Returns:\n",
    "        φ(n)\n",
    "    \"\"\"\n",
    "    return (p - 1) * (q - 1)\n",
    "\n",
    "# Prueba\n",
    "print(\"=== Función φ (Euler) ===\")\n",
    "p, q = 61, 53\n",
    "phi = generate_phi(p, q)\n",
    "print(f\"p = {p}, q = {q}\")\n",
    "print(f\"φ({p}×{q}) = ({p}-1)×({q}-1) = {phi}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_keys(p: int, q: int, e: int) -> tuple[tuple[int, int], tuple[int, int]]:\n",
    "    \"\"\"\n",
    "    Genera claves pública y privada RSA.\n",
    "    \n",
    "    Args:\n",
    "        p, q: Números primos distintos\n",
    "        e: Exponente público (debe cumplir gcd(e, φ(n)) = 1)\n",
    "    \n",
    "    Returns:\n",
    "        Tupla ((e, n), (d, n)) con clave pública y privada\n",
    "    \n",
    "    Raises:\n",
    "        ValueError: Si las condiciones RSA no se cumplen\n",
    "    \"\"\"\n",
    "    # Validaciones\n",
    "    if p == q:\n",
    "        raise ValueError(\"p y q deben ser distintos\")\n",
    "    \n",
    "    # Calcular n y φ(n)\n",
    "    n = p * q\n",
    "    phi = generate_phi(p, q)\n",
    "    \n",
    "    # Verificar que e sea válido\n",
    "    if not (1 < e < phi):\n",
    "        raise ValueError(f\"e debe estar en el rango (1, {phi})\")\n",
    "    \n",
    "    if gcd(e, phi) != 1:\n",
    "        raise ValueError(f\"gcd(e, φ(n)) = gcd({e}, {phi}) ≠ 1\")\n",
    "    \n",
    "    # Calcular d (inverso modular de e)\n",
    "    d = modular_inverse(e, phi)\n",
    "    \n",
    "    # Verificar que (e * d) mod φ(n) = 1\n",
    "    assert (e * d) % phi == 1, \"Error en cálculo de d\"\n",
    "    \n",
    "    public_key = (e, n)\n",
    "    private_key = (d, n)\n",
    "    \n",
    "    return public_key, private_key\n",
    "\n",
    "# Prueba con el ejemplo clásico\n",
    "print(\"\\n=== Generación de Claves RSA ===\")\n",
    "p, q, e = 61, 53, 17\n",
    "\n",
    "print(f\"Parámetros:\")\n",
    "print(f\"  p = {p} (primo)\")\n",
    "print(f\"  q = {q} (primo)\")\n",
    "print(f\"  e = {e} (exponente público)\")\n",
    "\n",
    "public_key, private_key = generate_keys(p, q, e)\n",
    "\n",
    "print(f\"\\nClaves generadas:\")\n",
    "print(f\"  Clave pública:  (e={public_key[0]}, n={public_key[1]})\")\n",
    "print(f\"  Clave privada:  (d={private_key[0]}, n={private_key[1]})\")\n",
    "\n",
    "# Verificación\n",
    "n = public_key[1]\n",
    "d = private_key[0]\n",
    "phi = generate_phi(p, q)\n",
    "print(f\"\\nVerificaciones:\")\n",
    "print(f\"  n = p×q = {p}×{q} = {n} ✓\")\n",
    "print(f\"  φ(n) = {phi} ✓\")\n",
    "print(f\"  (e×d) mod φ(n) = ({e}×{d}) mod {phi} = {(e*d) % phi} ✓\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "\n",
    "## 4️⃣ Cifrado y Descifrado RSA\n",
    "\n",
    "### 📖 Teoría\n",
    "\n",
    "**Cifrado**: $C = M^e \\mod n$\n",
    "\n",
    "**Descifrado**: $M = C^d \\mod n$\n",
    "\n",
    "**Teorema fundamental**: $(M^e)^d \\equiv M \\pmod{n}$ (por el Teorema de Euler)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def rsa_encrypt(message: int, e: int, n: int) -> int:\n",
    "    \"\"\"\n",
    "    Cifra un mensaje usando RSA.\n",
    "    \n",
    "    Args:\n",
    "        message: Mensaje en claro (entero < n)\n",
    "        e: Exponente público\n",
    "        n: Módulo público\n",
    "    \n",
    "    Returns:\n",
    "        Mensaje cifrado C = M^e mod n\n",
    "    \"\"\"\n",
    "    if message >= n:\n",
    "        raise ValueError(f\"Mensaje {message} debe ser < n ({n})\")\n",
    "    \n",
    "    return modular_exponentiation(message, e, n)\n",
    "\n",
    "\n",
    "def rsa_decrypt(cipher: int, d: int, n: int) -> int:\n",
    "    \"\"\"\n",
    "    Descifra un mensaje usando RSA.\n",
    "    \n",
    "    Args:\n",
    "        cipher: Mensaje cifrado\n",
    "        d: Exponente privado\n",
    "        n: Módulo público\n",
    "    \n",
    "    Returns:\n",
    "        Mensaje original M = C^d mod n\n",
    "    \"\"\"\n",
    "    return modular_exponentiation(cipher, d, n)\n",
    "\n",
    "# Pruebas\n",
    "print(\"=== Cifrado y Descifrado RSA ===\")\n",
    "\n",
    "# Usar claves generadas anteriormente\n",
    "e, n = public_key\n",
    "d, _ = private_key\n",
    "\n",
    "# Cifrar letra 'A' (código 65)\n",
    "M = 65\n",
    "print(f\"\\nMensaje original: M = {M} (letra 'A')\")\n",
    "\n",
    "C = rsa_encrypt(M, e, n)\n",
    "print(f\"Mensaje cifrado:  C = {M}^{e} mod {n} = {C}\")\n",
    "\n",
    "M2 = rsa_decrypt(C, d, n)\n",
    "print(f\"Mensaje descifrado: M = {C}^{d} mod {n} = {M2}\")\n",
    "\n",
    "print(f\"\\n✓ Verificación: M == M2 ? {M == M2}\")\n",
    "\n",
    "# Más ejemplos\n",
    "print(\"\\n\" + \"=\"*70)\n",
    "test_messages = [1, 100, 500, 1000, 3232]\n",
    "print(f\"\\n{'M':<10} {'C':<10} {'M recuperado':<15} {'Correcto?':<10}\")\n",
    "print(\"-\"*50)\n",
    "for m in test_messages:\n",
    "    c = rsa_encrypt(m, e, n)\n",
    "    m_recovered = rsa_decrypt(c, d, n)\n",
    "    print(f\"{m:<10} {c:<10} {m_recovered:<15} {'✓' if m == m_recovered else '✗':<10}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "\n",
    "## 5️⃣ Aplicación RSA con Alfabeto A-Z\n",
    "\n",
    "### 📖 Teoría\n",
    "\n",
    "Para cifrar texto:\n",
    "1. **Mapear**: A=00, B=01, ..., Z=25 (pares de dígitos)\n",
    "2. **Agrupar**: Formar bloques < n\n",
    "3. **Cifrar**: Cada bloque independientemente\n",
    "4. **Guardar**: Número de letras por bloque (para reconstrucción)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def text_to_numbers(text: str) -> str:\n",
    "    \"\"\"\n",
    "    Convierte texto a representación numérica.\n",
    "    A=00, B=01, ..., Z=25\n",
    "    \n",
    "    Args:\n",
    "        text: Texto en mayúsculas (A-Z)\n",
    "    \n",
    "    Returns:\n",
    "        String numérico con pares de dígitos\n",
    "    \"\"\"\n",
    "    text = text.upper()\n",
    "    numeric_str = \"\"\n",
    "    \n",
    "    for char in text:\n",
    "        if 'A' <= char <= 'Z':\n",
    "            value = ord(char) - ord('A')\n",
    "            numeric_str += f\"{value:02d}\"\n",
    "    \n",
    "    return numeric_str\n",
    "\n",
    "\n",
    "def numbers_to_text(numeric_str: str) -> str:\n",
    "    \"\"\"\n",
    "    Convierte representación numérica de vuelta a texto.\n",
    "    \n",
    "    Args:\n",
    "        numeric_str: String de dígitos (pares)\n",
    "    \n",
    "    Returns:\n",
    "        Texto reconstruido\n",
    "    \"\"\"\n",
    "    text = \"\"\n",
    "    \n",
    "    for i in range(0, len(numeric_str), 2):\n",
    "        if i + 1 < len(numeric_str):\n",
    "            pair = numeric_str[i:i+2]\n",
    "            value = int(pair)\n",
    "            if 0 <= value <= 25:\n",
    "                text += chr(ord('A') + value)\n",
    "    \n",
    "    return text\n",
    "\n",
    "# Pruebas\n",
    "print(\"=== Conversión Texto ↔ Números ===\")\n",
    "test_text = \"HELLO\"\n",
    "numeric = text_to_numbers(test_text)\n",
    "recovered = numbers_to_text(numeric)\n",
    "\n",
    "print(f\"Texto original:   {test_text}\")\n",
    "print(f\"Numérico:         {numeric}\")\n",
    "print(f\"Texto recuperado: {recovered}\")\n",
    "print(f\"\\n✓ Correcto: {test_text == recovered}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def split_into_blocks(numeric_str: str, n: int) -> list[tuple[int, int]]:\n",
    "    \"\"\"\n",
    "    Divide la cadena numérica en bloques < n.\n",
    "    Usa algoritmo greedy para maximizar tamaño de bloques.\n",
    "    \n",
    "    Args:\n",
    "        numeric_str: Cadena numérica\n",
    "        n: Módulo RSA\n",
    "    \n",
    "    Returns:\n",
    "        Lista de tuplas (valor_bloque, num_letras)\n",
    "    \"\"\"\n",
    "    blocks = []\n",
    "    i = 0\n",
    "    \n",
    "    while i < len(numeric_str):\n",
    "        current_block = \"\"\n",
    "        letters_count = 0\n",
    "        \n",
    "        while i < len(numeric_str):\n",
    "            if i + 1 < len(numeric_str):\n",
    "                next_pair = numeric_str[i:i+2]\n",
    "                test_block = current_block + next_pair\n",
    "                test_value = int(test_block)\n",
    "                \n",
    "                if test_value < n:\n",
    "                    current_block = test_block\n",
    "                    letters_count += 1\n",
    "                    i += 2\n",
    "                else:\n",
    "                    break\n",
    "            else:\n",
    "                break\n",
    "        \n",
    "        if current_block:\n",
    "            blocks.append((int(current_block), letters_count))\n",
    "    \n",
    "    return blocks\n",
    "\n",
    "# Prueba\n",
    "print(\"\\n=== División en Bloques ===\")\n",
    "numeric = text_to_numbers(\"HELLO\")\n",
    "n = 3233\n",
    "blocks = split_into_blocks(numeric, n)\n",
    "\n",
    "print(f\"Cadena numérica: {numeric}\")\n",
    "print(f\"Módulo n: {n}\")\n",
    "print(f\"\\nBloques generados:\")\n",
    "for i, (value, num_letters) in enumerate(blocks, 1):\n",
    "    print(f\"  Bloque {i}: {value:4d} ({num_letters} letras) < {n}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def encrypt_message(plaintext: str, e: int, n: int) -> list[tuple[int, int]]:\n",
    "    \"\"\"\n",
    "    Cifra un mensaje de texto completo.\n",
    "    \n",
    "    Args:\n",
    "        plaintext: Mensaje en texto claro\n",
    "        e: Exponente público\n",
    "        n: Módulo público\n",
    "    \n",
    "    Returns:\n",
    "        Lista de tuplas (bloque_cifrado, num_letras)\n",
    "    \"\"\"\n",
    "    # Convertir a números\n",
    "    numeric_str = text_to_numbers(plaintext)\n",
    "    \n",
    "    # Dividir en bloques\n",
    "    blocks = split_into_blocks(numeric_str, n)\n",
    "    \n",
    "    # Cifrar cada bloque\n",
    "    encrypted_blocks = []\n",
    "    for block_value, num_letters in blocks:\n",
    "        cipher_value = rsa_encrypt(block_value, e, n)\n",
    "        encrypted_blocks.append((cipher_value, num_letters))\n",
    "    \n",
    "    return encrypted_blocks\n",
    "\n",
    "\n",
    "def decrypt_message(encrypted_blocks: list[tuple[int, int]], d: int, n: int) -> str:\n",
    "    \"\"\"\n",
    "    Descifra un mensaje completo.\n",
    "    \n",
    "    Args:\n",
    "        encrypted_blocks: Lista de tuplas (bloque_cifrado, num_letras)\n",
    "        d: Exponente privado\n",
    "        n: Módulo público\n",
    "    \n",
    "    Returns:\n",
    "        Mensaje en texto claro\n",
    "    \"\"\"\n",
    "    numeric_str = \"\"\n",
    "    \n",
    "    for cipher_value, num_letters in encrypted_blocks:\n",
    "        # Descifrar bloque\n",
    "        block_value = rsa_decrypt(cipher_value, d, n)\n",
    "        \n",
    "        # Reconstruir con padding (2 dígitos por letra)\n",
    "        expected_length = num_letters * 2\n",
    "        block_str = str(block_value).zfill(expected_length)\n",
    "        \n",
    "        numeric_str += block_str\n",
    "    \n",
    "    # Convertir de vuelta a texto\n",
    "    return numbers_to_text(numeric_str)\n",
    "\n",
    "# Prueba completa\n",
    "print(\"\\n\" + \"=\"*70)\n",
    "print(\"=== Cifrado/Descifrado de Mensajes Completos ===\")\n",
    "print(\"=\"*70)\n",
    "\n",
    "plaintext = \"HELLO\"\n",
    "e, n = public_key\n",
    "d, _ = private_key\n",
    "\n",
    "print(f\"\\nMensaje original: {plaintext}\")\n",
    "\n",
    "# Cifrar\n",
    "cipher_blocks = encrypt_message(plaintext, e, n)\n",
    "print(f\"\\nBloques cifrados:\")\n",
    "for i, (cipher, num_letters) in enumerate(cipher_blocks, 1):\n",
    "    print(f\"  Bloque {i}: {cipher} ({num_letters} letras)\")\n",
    "\n",
    "# Descifrar\n",
    "decrypted = decrypt_message(cipher_blocks, d, n)\n",
    "print(f\"\\nMensaje descifrado: {decrypted}\")\n",
    "\n",
    "print(f\"\\n✓ Verificación: {plaintext} == {decrypted} ? {plaintext == decrypted}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "\n",
    "## 6️⃣ Casos de Prueba y Validación\n",
    "\n",
    "### Pruebas Exhaustivas"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"=\"*70)\n",
    "print(\"SUITE DE PRUEBAS COMPLETA\")\n",
    "print(\"=\"*70)\n",
    "\n",
    "# Configuración de prueba\n",
    "p, q, e = 61, 53, 17\n",
    "public_key, private_key = generate_keys(p, q, e)\n",
    "e, n = public_key\n",
    "d, _ = private_key\n",
    "\n",
    "# Lista de mensajes de prueba\n",
    "test_messages = [\n",
    "    \"A\",\n",
    "    \"HELLO\",\n",
    "    \"ATTACK\",\n",
    "    \"CRYPTOGRAPHY\",\n",
    "    \"THEQUICKBROWNFOX\",\n",
    "    \"Z\" * 10  # Repetición\n",
    "]\n",
    "\n",
    "print(f\"\\nClaves: e={e}, d={d}, n={n}\")\n",
    "print(f\"\\n{'Mensaje':<20} {'Bloques cifrados':<30} {'Recuperado':<20} {'OK?'}\")\n",
    "print(\"-\"*80)\n",
    "\n",
    "all_passed = True\n",
    "\n",
    "for msg in test_messages:\n",
    "    try:\n",
    "        # Cifrar\n",
    "        encrypted = encrypt_message(msg, e, n)\n",
    "        cipher_str = str([c for c, _ in encrypted])\n",
    "        \n",
    "        # Descifrar\n",
    "        decrypted = decrypt_message(encrypted, d, n)\n",
    "        \n",
    "        # Verificar\n",
    "        passed = (msg == decrypted)\n",
    "        all_passed = all_passed and passed\n",
    "        \n",
    "        status = \"✓\" if passed else \"✗\"\n",
    "        print(f\"{msg:<20} {cipher_str:<30} {decrypted:<20} {status}\")\n",
    "        \n",
    "    except Exception as ex:\n",
    "        print(f\"{msg:<20} ERROR: {str(ex)}\")\n",
    "        all_passed = False\n",
    "\n",
    "print(\"-\"*80)\n",
    "print(f\"\\n{'✓✓✓ TODAS LAS PRUEBAS PASARON ✓✓✓' if all_passed else '✗✗✗ ALGUNAS PRUEBAS FALLARON ✗✗✗'}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Análisis de Seguridad Básico"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\n\" + \"=\"*70)\n",
    "print(\"ANÁLISIS DE PARÁMETROS RSA\")\n",
    "print(\"=\"*70)\n",
    "\n",
    "print(f\"\\n1. Tamaño de clave:\")\n",
    "print(f\"   n = {n}\")\n",
    "print(f\"   Bits: {n.bit_length()} bits\")\n",
    "print(f\"   Nota: Para seguridad real, se necesitan al menos 2048 bits\")\n",
    "\n",
    "print(f\"\\n2. Primos utilizados:\")\n",
    "print(f\"   p = {p}\")\n",
    "print(f\"   q = {q}\")\n",
    "print(f\"   p ≠ q: {p != q} ✓\")\n",
    "\n",
    "print(f\"\\n3. Exponente público:\")\n",
    "print(f\"   e = {e}\")\n",
    "phi = (p-1) * (q-1)\n",
    "print(f\"   φ(n) = {phi}\")\n",
    "print(f\"   gcd(e, φ(n)) = {gcd(e, phi)} ✓\")\n",
    "\n",
    "print(f\"\\n4. Exponente privado:\")\n",
    "print(f\"   d = {d}\")\n",
    "print(f\"   (e × d) mod φ(n) = {(e * d) % phi} ✓\")\n",
    "\n",
    "print(f\"\\n5. Capacidad de mensajes:\")\n",
    "max_letters = 0\n",
    "test_block = \"\"\n",
    "while int(test_block + \"25\") < n:  # 'Z' = 25\n",
    "    test_block += \"25\"\n",
    "    max_letters += 1\n",
    "print(f\"   Máximo de letras por bloque: {max_letters}\")\n",
    "print(f\"   Valor máximo de bloque: {int(test_block)}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "\n",
    "## 📊 Resumen y Conclusiones\n",
    "\n",
    "### Algoritmos Implementados\n",
    "\n",
    "✅ **Euclides y Euclides Extendido**: Cálculo de MCD e inversos modulares\n",
    "\n",
    "✅ **Exponenciación Modular Rápida**: Algoritmo eficiente O(log n)\n",
    "\n",
    "✅ **Generación de Claves RSA**: Validación completa de parámetros\n",
    "\n",
    "✅ **Cifrado/Descifrado**: Implementación correcta del esquema RSA\n",
    "\n",
    "✅ **Manejo de Texto**: Sistema de bloques con padding correcto\n",
    "\n",
    "### Conceptos Clave Aprendidos\n",
    "\n",
    "1. **La seguridad de RSA** se basa en la dificultad de factorizar n = p×q\n",
    "2. **El Teorema de Euler** garantiza que (M^e)^d ≡ M (mod n)\n",
    "3. **La exponenciación modular** es crítica para la eficiencia\n",
    "4. **El padding** es esencial para reconstruir mensajes correctamente\n",
    "\n",
    "### Limitaciones de esta Implementación\n",
    "\n",
    "⚠️ **Educativa**: No usar en producción\n",
    "- Primos pequeños (61, 53) son trivialmente factorizables\n",
    "- No hay padding criptográfico (OAEP)\n",
    "- No hay protección contra ataques de temporización\n",
    "- Para uso real: usar bibliotecas como `cryptography` o `PyCryptodome`\n",
    "\n",
    "---\n",
    "\n",
    "## 🎓 Ejercicios Adicionales\n",
    "\n",
    "1. Implementar un test de primalidad (Miller-Rabin)\n",
    "2. Generar primos aleatorios de mayor tamaño\n",
    "3. Comparar tiempos de ejecución con/sin exponenciación rápida\n",
    "4. Implementar padding PKCS#1 v1.5\n",
    "5. Cifrar/descifrar archivos pequeños\n",
    "\n",
    "---\n",
    "\n",
    "## 📚 Referencias\n",
    "\n",
    "- **Rivest, Shamir, Adleman** (1977): \"A Method for Obtaining Digital Signatures and Public-Key Cryptosystems\"\n",
    "- **Cormen et al.**: \"Introduction to Algorithms\" (Capítulo de Teoría de Números)\n",
    "- **RFC 8017**: PKCS #1: RSA Cryptography Specifications\n",
    "\n",
    "---\n",
    "\n",
    "*Laboratorio completado exitosamente* ✅"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}